In [1]:
import pygame
import random, time, sys, h5py
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
from tqdm import tqdm
from skimage.transform import resize

pygame 2.6.1 (SDL 2.28.4, Python 3.13.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
hf = h5py.File('../data.h5', 'r')
frames = torch.tensor(torch.from_numpy(hf["frame"][:]), dtype=torch.float) # (batch_size, 76, 119)
frames =  frames / torch.max(frames)
moves = torch.tensor(torch.from_numpy(hf["move"][:]), dtype=torch.long)

C:\Users\arthu\AppData\Local\Temp\ipykernel_5032\2225926222.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  frames = torch.tensor(torch.from_numpy(hf["frame"][:]), dtype=torch.float) # (batch_size, 76, 119)
C:\Users\arthu\AppData\Local\Temp\ipykernel_5032\2225926222.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  moves = torch.tensor(torch.from_numpy(hf["move"][:]), dtype=torch.long)


In [15]:
moves.shape

torch.Size([40, 753])

In [27]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 3, kernel_size=(24, 30), stride=2)
        self.conv2 = nn.Conv2d(3, 1, kernel_size=(10, 10), stride=1)
        self.fc1 = nn.Linear(522, 228) 
        self.fc2 = nn.Linear(228, 753) 

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.flatten(x, start_dim=1)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [7]:
frames[0].unsqueeze(0).shape

torch.Size([1, 76, 104])

In [28]:
model = Model()
o = model.forward(frames[0].unsqueeze(0))
print(o.shape)

torch.Size([1, 18, 29])
torch.Size([1, 522])
torch.Size([1, 753])
